In [3]:
!pip install torch news-please numpy pandas datasets matplotlib

In [4]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

print(torch.__version__)

print(torch.randn(3,3))
print(torch.cuda.is_available())


c:\Users\ryanh\miniconda3\envs\frame-delta\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.9.1+cpu
tensor([[-1.2422,  0.5211,  0.1388],
        [-0.6414, -1.5446, -0.0359],
        [-0.2370, -1.5817, -1.2662]])
False


## Load the data

In [28]:
# this loads data as an apache arrow format, mapping to disk
data = load_dataset("copenlu/mm-framing")
data_raw = data
data = data['full']
data[1], type(data)

({'uuid': '0000442d-4ce2-4d1c-b654-5561c3cab3b7',
  'title': 'NJ community teams up to help repair historic cemetery damaged by August storms',
  'date_publish': '2023-08-23 22:25:00',
  'source_domain': 'www.cbsnews.com',
  'url': 'https://www.cbsnews.com/philadelphia/video/nj-community-teams-up-to-help-repair-historic-cemetery-damaged-by-august-storms/',
  'political_leaning': 'left_lean',
  'text-topic': 'Historic Cemetery Damage and Restoration',
  'text-topic-exp': "The article discusses the damage caused to a historic cemetery by storms and the community's efforts to repair it. The cemetery is significant as it is the final resting place of several black Civil War veterans.",
  'text-entity-name': "Woodstown church's historic cemetery",
  'text-entity-sentiment': 'Positive',
  'text-entity-sentiment-exp': 'The article describes efforts to repair and clean up the cemetery, indicating a positive sentiment towards the entity.',
  'text-generic-frame': "['Economic', 'Health and safet

### Hydrating the urls test

In [9]:
from newsplease import NewsPlease
test_index = 21000

url = data[test_index]["url"]

article = NewsPlease.from_url(url)

print(article.maintext)
print(url)
data[test_index]

def get_article_text(url):
    text = NewsPlease.from_url(url)
    return(text)



At first, there is some fussing from my 10- and 12-year-old sons. “Will the water be too cold?”, “I don’t want to wear a wetsuit”, “Are there even any decent waves?”… But once their young surf instructor, Josh Mahony, appears with two boards under his arms and the kind of shoulders that suggest he could paddle south to Spain and back in his lunch break, there is only silence as they follow him into the waves.
Some hours later, having caught many waves including their first proper unbroken green ones – before today they’d only surfed more easily managed whitewater – it’s an effort to get them out.
We’ve come to Garretstown in County Cork, a picturesque stretch of sandy beach dotted with smooth grey, brown and purple pebbles. In the car park, an old man is selling honey, eggs and swedes from a picnic table, while all around us is verdant green countryside and cows, save a brutalist concrete toilet block, a yellow airstream pizza van and a wooden sauna sitting above the shoreline.
The boy

After checking several examples of the text, it seems that a large portion of the news are non-political. Seems to be a flaw with the original paper. 

TO DO: Filter the dataset to get just the truly 'political' news stories

TO DO: Check out the other dataset, the 'validation' set to see if it different in terms of its content

In [ ]:


data_raw['valid_framing_subset'][150]



{'uuid': '019a66c8-1d6f-42f7-ae2c-ade5b91d6b45',
 'title': 'Florida neighborhood terrorized after crocodile eats small dog',
 'date_publish': '2023-08-02 16:09:12',
 'source_domain': 'www.foxnews.com',
 'url': 'https://www.foxnews.com/us/florida-neighborhood-terrorized-crocodile-eats-small-dog',
 'political_leaning': 'right',
 'text-topic': 'Crocodile Attack',
 'text-topic-exp': "The article discusses an incident where a crocodile ate a small dog in a Florida canal, and it includes quotes from witnesses and the Florida Fish and Wildlife Conservation Commission's response to the incident.",
 'text-entity-name': 'Crocodile',
 'text-entity-sentiment': 'negative',
 'text-entity-sentiment-exp': 'The crocodile is portrayed as a dangerous and harmful entity that caused the death of a pet dog and caused concern among the residents of Satellite Beach, Florida.',
 'text-generic-frame': "['crime', 'security', 'public_op']",
 'text-generic-frame-exp': "The article discusses an incident of a crocod

So it seems that the validation set is similar in having non-political articles


For now, run with just a subset of the data. Ignore this issue. Parse later before full training runs.

In order to create a hydrated version of the data, bu keep the arrow datasets (which are immutable), then we use a .map() method

In [38]:
# Create a minimal dataset to process in this notebook with hydrated article text

import time
from newsplease import NewsPlease

def scrape_and_add(example):
    """Scrape article text from URL with error handling"""
    url = example['url']

    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)

        # Try to fetch the article
        article = NewsPlease.from_url(url, timeout=10)

        # Return the maintext if available, otherwise empty string
        if article and article.maintext:
            return {"article_text": article.maintext}
        else:
            print(f"No text found for: {url}")
            return {"article_text": ""}

    except Exception as e:
        # Handle 403s and other errors gracefully
        print(f"Error fetching {url}: {type(e).__name__} - {str(e)}")
        return {"article_text": ""}

# Create a small dataset for testing
df = data.shuffle(seed=42).select(range(25))

# Apply the scraping function to each row (batched=False means one at a time)
print("Starting to hydrate URLs...")
df = df.map(scrape_and_add, batched=False)
print("Hydration complete!")

# Check the result
df[3]['article_text']

Starting to hydrate URLs...


Map:   4%|▍         | 1/25 [00:00<00:12,  1.93 examples/s]

Error fetching https://www.cbsnews.com/news/trump-suing-abc-news-george-stephanopoulos-defamation/: TypeError - NewsPlease.from_url() got an unexpected keyword argument 'timeout'


Map:   8%|▊         | 2/25 [00:01<00:11,  1.96 examples/s]

Error fetching https://apnews.com/article/cowboys-malik-hooker-contract-5531a25a7aecda62883756a7270dfc89: TypeError - NewsPlease.from_url() got an unexpected keyword argument 'timeout'


...

Map:  96%|█████████▌| 24/25 [00:12<00:00,  1.98 examples/s]

Error fetching https://www.forbes.com/sites/johnnavin/2023/05/30/4-cheap-value-stocks-below-book-value-dividend-paying/: TypeError - NewsPlease.from_url() got an unexpected keyword argument 'timeout'


Map: 100%|██████████| 25/25 [00:12<00:00,  1.97 examples/s]

Error fetching https://www.foxbusiness.com/markets/home-depot-bulks-up-pro-business-18-25-billion-deal-building-products-supplier-srs: TypeError - NewsPlease.from_url() got an unexpected keyword argument 'timeout'
Hydration complete!


''

## Tokenize and encode

In [ ]:
df.keys()

data[4]['text'] = [0]


KeyError: 'text'

## Set Format

## Run sanity checks